In [208]:
# import library
import json
from copy import deepcopy

import numpy as np
import marmote.core as mc
import marmote.markovchain as mmc
import networkx as nx


In [209]:
# read jani file
jani_file = "die.jani"

with open(jani_file, 'r', encoding='utf-8') as f:
    jani_data = json.load(f)

In [210]:
# list of variables
variables = jani_data["variables"]
dim = len(variables)
varDict = {}
for k, var in enumerate(variables):
    varDict[var['name']] = k

print(varDict)

{'s': 0, 'd': 1}


In [211]:
def satisfyExpression(exp, node):
    if exp['op'] == '∧':
        return satisfyExpression(exp['left'], node) and satisfyExpression(exp['right'], node)
    idx = varDict[exp['left']]
    if exp['op'] == '=':
        return node[idx] == exp['right']
    elif exp['op'] == '<':
        return node[idx] < exp['right']
    elif exp['op'] == '>':
        return node[idx] > exp['right']

In [212]:
root = None
root = tuple(var['initial-value'] for var in variables)

openNode = [root]
closedNode = []

G = nx.DiGraph()
automata = jani_data["automata"]
edges = automata[0]["edges"]
while openNode:
    x = openNode.pop(0)
    for edge in edges:
        guard = edge['guard']['exp']
        if satisfyExpression(guard, x):
            closedNode.append(x)
            nbdest = 0
            for dest in edge["destinations"]:
                nbdest += 1
                print('nbdest : ',nbdest)
                newNode = deepcopy(x)
                for assignment in dest["assignments"]:
                    idx = varDict[assignment['ref']]
                    print('x: ', x)
                    newNode = newNode[:idx]+(assignment['value'],)+newNode[idx+1:]
                print('newNode: ', newNode)
                if not newNode in openNode and not newNode in closedNode:
                    openNode.append(newNode)
                try:
                    val = dest["probability"]["exp"]
                except KeyError:
                    val = 1
                print('val: ', val)
                G.add_edge(x, newNode, weight=val)

            break
        else:
            continue

print(closedNode)

nbdest :  1
x:  (0, 0)
newNode:  (1, 0)
val:  0.5
nbdest :  2
x:  (0, 0)
newNode:  (2, 0)
val:  0.5
nbdest :  1
x:  (1, 0)
newNode:  (3, 0)
val:  0.5
nbdest :  2
x:  (1, 0)
newNode:  (4, 0)
val:  0.5
nbdest :  1
x:  (2, 0)
newNode:  (5, 0)
val:  0.5
nbdest :  2
x:  (2, 0)
newNode:  (6, 0)
val:  0.5
nbdest :  1
x:  (3, 0)
newNode:  (1, 0)
val:  0.5
nbdest :  2
x:  (3, 0)
x:  (3, 0)
newNode:  (7, 1)
val:  0.5
nbdest :  1
x:  (4, 0)
x:  (4, 0)
newNode:  (7, 2)
val:  0.5
nbdest :  2
x:  (4, 0)
x:  (4, 0)
newNode:  (7, 3)
val:  0.5
nbdest :  1
x:  (5, 0)
x:  (5, 0)
newNode:  (7, 4)
val:  0.5
nbdest :  2
x:  (5, 0)
x:  (5, 0)
newNode:  (7, 5)
val:  0.5
nbdest :  1
x:  (6, 0)
newNode:  (2, 0)
val:  0.5
nbdest :  2
x:  (6, 0)
x:  (6, 0)
newNode:  (7, 6)
val:  0.5
nbdest :  1
x:  (7, 1)
newNode:  (7, 1)
val:  1
nbdest :  1
x:  (7, 2)
newNode:  (7, 2)
val:  1
nbdest :  1
x:  (7, 3)
newNode:  (7, 3)
val:  1
nbdest :  1
x:  (7, 4)
newNode:  (7, 4)
val:  1
nbdest :  1
x:  (7, 5)
newNode:  (7, 5)
va

In [213]:
nodes = sorted(list(G.nodes()))
edges = [(u,v, G[u][v]['weight']) for u, v in G.edges()]
len(edges)
len(nodes)

13

In [214]:
state_map = {state: idx for idx, state in enumerate(nodes)}
n = len(nodes)
P = mc.SparseMatrix(n)
if jani_data["type"] == "dtmc":
    P.set_type(mc.DISCRETE)
for u, v, weight in edges:
    P.setEntry(state_map[u], state_map[v], weight)

print(P)

[[0.000000e+00, 5.000000e-01, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
 [0.000000e+00, 0.000000e+00, 0.000000e+00, 5.000000e-01, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
 [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 5.000000e-01, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
 [0.000000e+00, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
 [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 5.000000e-01, 5.000000e-01, 0.000000e+00, 0.000000e+00, 0.000000e+00],
 [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.0

In [215]:
P.getEntry(7,7)

1.0

In [216]:

initial_prob = np.zeros(n)
initState = []
for variable in variables:
    initState.append(variable['initial-value'])
initial_prob[state_map[tuple(initState)]] = 1

initial = mc.DiscreteDistribution(np.array(list(state_map.values())), initial_prob)
print(initial)

Discrete distribution values { 0 1 2 3 4 5 6 7 8 9 10 11 12 } probas {        1        0        0        0        0        0        0        0        0        0        0        0        0 }


In [217]:
c = mmc.MarkovChain(P)
c.set_init_distribution(initial)
c.set_model_name(jani_data["name"])

In [218]:
simRes = c.SimulateChainDT(20, stats = False, traj = True, trace = False)
print(simRes.states())

[ 0  2  5 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11]


In [219]:
print( c.generator().toString( mc.FORMAT_NUMPY ) )

mc_matrix=np.array([
[0, 0.5, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0.5, 0.5, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0.5, 0.5, 0, 0, 0, 0, 0, 0],
[0, 0.5, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0.5, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0.5, 0],
[0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
], dtype=float)



In [220]:
pi3 = c.TransientDistributionDT(3)
print(pi3)

Discrete distribution values { 0  1  2  3  4  5  6  7  8  9  10  11  12  } probas {        0    0.125    0.125        0        0        0        0    0.125    0.125    0.125    0.125    0.125    0.125 }


In [221]:
pista = c.StationaryDistribution()
print(pista)

Discrete distribution values { 0  1  2  3  4  5  6  7  8  9  10  11  12  } probas {        0 7.451e-09 7.451e-09        0        0        0        0   0.1667   0.1667   0.1667   0.1667   0.1667   0.1667 }


In [222]:
c.set_init_distribution(pista)
dis = c.TransientDistributionDT(1)
print("Distance between pi and pi.P : ", mc.DiscreteDistribution.DistanceL1(pista, dis))

Distance between pi and pi.P :  2.9802322387695312e-08


In [223]:
pista2 = c.StationaryDistributionRLGL(100, 1e-10, mc.UniformDiscreteDistribution(0,2), False)
mc.DiscreteDistribution.DistanceL1(pista, pista2)

Warning in MarkovChain::StationaryDistributionRLGL(): incorrect number of values for initial distribution (expected 13 <> 3 got).
Uniform assumed.


2.9724712080611034e-08

In [224]:
simRes = c.SimulateChainDT(10, stats = False, traj = True, trace = False)

In [225]:
simRes.Diagnose()

# Simulation result
# Time type: discrete
# Keeps a trajectory of size 11
# DT trajectory size: 11
# CT trajectory size: 0
# Last state: 8
# Last time:  10


In [226]:
print(simRes.states())

[8 8 8 8 8 8 8 8 8 8 8]
